# **Imports**

In [8]:
import os
import sys
import tqdm
from pathlib import Path
from tqdm.notebook import trange, tqdm
from warnings import filterwarnings
from dotenv import load_dotenv

load_dotenv()
filterwarnings("ignore")

PATH = os.getcwd()
PROJECT = str(Path(PATH).parents[0])
GOOGLE_MAPS_API = os.getenv("GOOGLE_MAPS_API")

In [15]:
import numpy as np
import pandas as pd
import ampligraph
import requests
from ampligraph.datasets import load_from_csv
import json
from urllib.parse import quote_plus

ampligraph.__version__

'1.4.0'

## **Preprocess**

In [126]:
# url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Anejo&inputtype=textquery&key={GOOGLE_MAPS_API}"

# respon = requests.get(url)
# json.loads(respon.text)
# url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJr_tCsVg1K4gREHA5C-YHJys&fields=types,rating,price_level&key={GOOGLE_MAPS_API}" #
# respon = requests.get(url)
# json.loads(respon.text)

In [79]:
def get_place_id(place):
    place = quote_plus(place)
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place}+Toronto&inputtype=textquery&key={GOOGLE_MAPS_API}"
    response = requests.get(url)
    try:
        return json.loads(response.text)["candidates"][0]["place_id"]
    except:
        return None

In [82]:
file = f"{PROJECT}/data/maps/Want to go.csv"
df = pd.read_csv(file)[["Title"]]
df["place_id"] = df.Title.map(lambda x: get_place_id(x))
df.to_csv(file, index=False)

file = f"{PROJECT}/data/maps/Favorite places.csv"
df = pd.read_csv(file)[["Title"]]
df["place_id"] = df.Title.map(lambda x: get_place_id(x))
df.to_csv(file, index=False)

,Title,place_id
0,Shook,ChIJ8eB4-940K4gR2K8ckRlXgmY
1,Sofia Restaurant & Bar,ChIJJWuXY6U0K4gRRc0QwSyBsN8
2,Sunnyside Beach,ChIJUZPdgcg1K4gRAqHCKWed9bM
3,Stock T.C,ChIJ10YjyKUzK4gRY_JulT4V3SM
4,Colossal Cookie Company,None


In [131]:
file = f"{PROJECT}/data/maps/Saved Places.json"
df = pd.read_json(file)
df = pd.DataFrame(list(df.features))
df["Title"] = df["properties"].apply(lambda x: x.get("Title"))
df["place_id"] = df.Title.map(lambda x: get_place_id(x))
file = f"{PROJECT}/data/maps/Saved places.csv"
df[["Title", "place_id"]].to_csv(file, index=False)

In [132]:
df

,geometry,properties,type,Title,place_id
0,"{'coordinates': [-79.3983552, 43.6436227], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Marben,ChIJ7xo6It80K4gR7DCOJQPoG5c
1,"{'coordinates': [-79.3976425, 43.702135], 'typ...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Mariachi's Restaurant,ChIJ2azwrT4zK4gRu1wznQVfdEs
2,"{'coordinates': [-79.4015223, 43.6529421], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,The Boat Restaurant,ChIJyfwvjcI0K4gR1knTSasoI-E
3,"{'coordinates': [-79.400137, 43.6528], 'type':...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,El Rey Mezcal Bar,ChIJC7ZkAsM0K4gRXV1uVuN0T-s
4,"{'coordinates': [-79.3921423, 43.6557344], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Krispy Kreme Café,ChIJD4ldGsY0K4gRAqNxz_S73jE
...,...,...,...,...,...
291,"{'coordinates': [-79.3805639, 43.6642444], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Crews & Tangos Restaurant & Bars,ChIJtQsiS7M0K4gRgR8DOWCnDQo
292,"{'coordinates': [-79.3876304, 43.6485905], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Cake Nightclub,ChIJ0ZKQwNE0K4gRoXARYglzPEk
293,"{'coordinates': [-79.4006213, 43.6582582], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Comfort Zone,ChIJu7jbC7U1K4gR4DKZBWOEBqM
294,"{'coordinates': [-79.3805207, 43.6426286], 'ty...",{'Google Maps URL': 'http://maps.google.com/?q...,Feature,"Maple Leaf Square, Bremner Blvd, Toronto, ON M...",ChIJaf1cydQ0K4gRWI0gKkEZETY


### **Create Knowledge Graphs**

In [ ]:
file = f"{PROJECT}/data/maps/Want to go.csv"
file = f"{PROJECT}/data/maps/Saved places.csv"
file = f"{PROJECT}/data/maps/Favorite places.csv"

In [ ]:
entities = np.unique(np.concatenate([X[:, 0], X[:, 2]]))
relations = np.unique(X[:, 1])
print(relations, entities)

In [4]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_test = train_test_split_no_unseen(X, test_size=100) 
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (3075, 3)
Test set size:  (100, 3)


In [5]:
from ampligraph.latent_features import ComplEx

In [6]:
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [7]:
positives_filter = X
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

Average ComplEx Loss:   0.018132: 100%|██████████| 200/200 [03:13<00:00,  1.03epoch/s]


In [9]:
from ampligraph.latent_features import save_model, restore_model
save_model(model, './best_model.pkl')

In [10]:
from ampligraph.evaluation import evaluate_performance
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 100/100 [00:01<00:00, 52.67it/s]


MRR: 0.44
Hits@10: 0.57
Hits@3: 0.46
Hits@1: 0.36


In [12]:
X_unseen = np.array([
    ['Jorah Mormont', 'SPOUSE', 'Daenerys Targaryen'],
    ['Tyrion Lannister', 'SPOUSE', 'Missandei'],
    ["King's Landing", 'SEAT_OF', 'House Lannister of Casterly Rock'],
    ['Sansa Stark', 'SPOUSE', 'Petyr Baelish'],
    ['Daenerys Targaryen', 'SPOUSE', 'Jon Snow'],
    ['Daenerys Targaryen', 'SPOUSE', 'Craster'],
    ['House Stark of Winterfell', 'IN_REGION', 'The North'],
    ['House Stark of Winterfell', 'IN_REGION', 'Dorne'],
    ['House Tyrell of Highgarden', 'IN_REGION', 'Beyond the Wall'],
    ['Brandon Stark', 'ALLIED_WITH', 'House Stark of Winterfell'],
    ['Brandon Stark', 'ALLIED_WITH', 'House Lannister of Casterly Rock'],    
    ['Rhaegar Targaryen', 'PARENT_OF', 'Jon Snow'],
    ['House Hutcheson', 'SWORN_TO', 'House Tyrell of Highgarden'],
    ['Daenerys Targaryen', 'ALLIED_WITH', 'House Stark of Winterfell'],
    ['Daenerys Targaryen', 'ALLIED_WITH', 'House Lannister of Casterly Rock'],
    ['Jaime Lannister', 'PARENT_OF', 'Myrcella Baratheon'],
    ['Robert I Baratheon', 'PARENT_OF', 'Myrcella Baratheon'],
    ['Cersei Lannister', 'PARENT_OF', 'Myrcella Baratheon'],
    ['Cersei Lannister', 'PARENT_OF', 'Brandon Stark'],
    ["Tywin Lannister", 'PARENT_OF', 'Jaime Lannister'],
    ["Missandei", 'SPOUSE', 'Grey Worm'],
    ["Brienne of Tarth", 'SPOUSE', 'Jaime Lannister']
])

unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)
scores = model.predict(X_unseen)

from scipy.special import expit
probs = expit(scores)

pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

100%|██████████| 22/22 [00:00<00:00, 35.85it/s]


,statement,rank,score,prob
10,Brandon Stark ALLIED_WITH House Lannister of C...,3998,-2.925928,0.050887
18,Cersei Lannister PARENT_OF Brandon Stark,4079,-2.160656,0.103340
0,Jorah Mormont SPOUSE Daenerys Targaryen,3319,-0.859073,0.297533
1,Tyrion Lannister SPOUSE Missandei,2977,-0.542985,0.367493
5,Daenerys Targaryen SPOUSE Craster,2936,-0.535724,0.369183
7,House Stark of Winterfell IN_REGION Dorne,2504,-0.311248,0.422810
15,Jaime Lannister PARENT_OF Myrcella Baratheon,2817,-0.266177,0.433846
11,Rhaegar Targaryen PARENT_OF Jon Snow,3374,-0.232267,0.442193
4,Daenerys Targaryen SPOUSE Jon Snow,2309,-0.099999,0.475021
21,Brienne of Tarth SPOUSE Jaime Lannister,1938,-0.039280,0.490181
